# Implmentation of LSTM and GRU

The maximum accuracy achived on MNIST for both LSTM and GRU is 97%. 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

### Learning curve for GRU and LSTM

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
import glob,os
import math
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

cuda = True if torch.cuda.is_available() else False
    
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor    

torch.manual_seed(125)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(125)


In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')

In [ ]:
base_path = "/content/gdrive/MyDrive/Infinite hand/alphabet"

In [ ]:
#read train dataset
train_dir = base_path
train_names = glob.glob(os.path.join(train_dir, '*') )
train_numpy = []
train_name = []
for image_path in train_names:
    #print(image_path)
    alphabet = image_path[-7]
    alphabet = ord(alphabet)-65
    alp_arr = [alphabet] * 95
    im = np.load(image_path)
    train_numpy.append(im[5:])
    train_name.append(alp_arr)
    print(image_path[-7],alphabet,len(train_numpy))
train_numpy = np.array(train_numpy).reshape(([-1,4,400])).astype('float')
train_numpy = np.transpose(train_numpy, (0, 2, 1))
train_name = np.array(train_name).reshape((-1))
np.save('train',train_numpy)
np.save('train_alphabet',train_name)
print(train_numpy.shape)
print(train_name.shape)

In [ ]:
for i in range(len(train_numpy)):
    for k in range(3):
        max_num = np.max(train_numpy[i,:,k])
        min_num = np.min(train_numpy[i,:,k])
        if((max_num - min_num)!= 0):
            train_numpy[i,:,k] = (train_numpy[i,:,k]-min_num)/(max_num - min_num)
        else:
            train_numpy[i,:,k] = 0

In [ ]:
class CustomTensorDataset(TensorDataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors,alphabet):
        self.tensors = tensors
        self.alphabet = alphabet
        
    def __getitem__(self, index):
        x = self.tensors[index]
        alph = self.alphabet[index]
        return x,alph.float() 

    def __len__(self):
        return len(self.tensors)


In [ ]:
batch_size = 64
data = torch.from_numpy(train_numpy)
name = torch.from_numpy(train_name)
full_dataset = CustomTensorDataset(data,name)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
test_sampler = RandomSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

In [ ]:
train_dataset[3]

## GRU cell implmentation

In [ ]:
class GRUCell(nn.Module):

    """
    An implementation of GRUCell.

    """

    def __init__(self, input_size, hidden_size, bias=True):
        super(GRUCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 3 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 3 * hidden_size, bias=bias)
        self.reset_parameters()



    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)
    
    def forward(self, x, hidden):
        
        x = x.view(-1, x.size(1))
        
        gate_x = self.x2h(x) 
        gate_h = self.h2h(hidden)
        
        gate_x = gate_x.squeeze()
        gate_h = gate_h.squeeze()
        if(len(gate_x.size())<=1):
            gate_x = gate_x.unsqueeze(0)
            gate_h = gate_h.unsqueeze(0)             
        i_r, i_i, i_n = gate_x.chunk(3, 1)
        h_r, h_i, h_n = gate_h.chunk(3, 1)
        
        
        resetgate = F.sigmoid(i_r + h_r)
        inputgate = F.sigmoid(i_i + h_i)
        newgate = F.tanh(i_n + (resetgate * h_n))
        
        hy = newgate + inputgate * (hidden - newgate)
        
        
        return hy


In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(GRUModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim
         
        # Number of hidden layers
        self.layer_dim = layer_dim
         
       
        self.gru_cell = GRUCell(input_dim, hidden_dim, layer_dim)
        
        
        self.fc = nn.Linear(hidden_dim, output_dim)
     
    
    
    def forward(self, x):
        
        # Initialize hidden state with zeros
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        #print(x.shape,"x.shape")100, 28, 28
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
         
       
        outs = []
        
        hn = h0[0,:,:]
        
        for seq in range(x.size(1)):
            hn = self.gru_cell(x[:,seq,:], hn) 
            outs.append(hn)
            

        out = outs[-1].squeeze()
        
        out = self.fc(out) 
        # out.size() --> 100, 10
        return out
 

## LSTM cell implementation

In [ ]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 4
hidden_dim = 128
layer_dim = 1  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 26
 
# model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
model = GRUModel(input_dim, hidden_dim, layer_dim, output_dim)

#######################
#  USE GPU FOR MODEL  #
#######################
 
if torch.cuda.is_available():
    model.cuda()
     
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()
 
'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1
 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  


In [ ]:
from torchvision import models
from torchsummary import summary
summary(model, (400, 4))

In [ ]:
'''
STEP 7: TRAIN THE MODEL
'''
 
# Number of steps to unroll
seq_dim = 400
batch_size = 32
num_epochs = 6000
loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i,(images, labels) in enumerate(train_dataloader):
        # Load images as Variable
        #######################
        #  USE GPU FOR MODEL  #
        #######################
          
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda().float())
            #print(images.size())
            #print(labels)
            #print(images)
            #print(type(images))
            #print(type(labels))
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, seq_dim, input_dim).float() )
            labels = Variable(labels)
        #print(labels)
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images)
        #print(outputs.dtype)
        #print(labels.to(torch.int64).dtype)
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels.to(torch.int64))

        if torch.cuda.is_available():
            loss.cuda()

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
        
        loss_list.append(loss.item())
        iter += 1
         
        if iter % 5 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for (images, labels) in test_dataloader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    images = Variable(images.view(-1, seq_dim, input_dim).cuda().float())
                else:
                    images = Variable(images.view(-1 , seq_dim, input_dim).float())
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                # Total correct predictions
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
             
            accuracy = 100 * correct / total
             
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

In [ ]:
torch.save(model, 'all_final.pt')

In [ ]:
torch.save(model.state_dict(), 'all_weight.pt')

In [ ]:
#model = torch.load('all.pt').cuda()
model.load_state_dict(torch.load("all_weight.pt")) 

In [ ]:
import time

start = time.time()
outputs = model(images)
end = time.time()
print(end - start)

In [ ]:
import serial
import mouse
import time
model.load_state_dict(torch.load("all_weight.pt")) 
ser = serial.Serial("COM11", 9600)
last_band = 0
while True:
    read_bluetooth=ser.readline()
    #print(read_bluetooth[0],read_bluetooth[1])
    #print(read_bluetooth.shape())
    try:
        read_value_x = int(read_bluetooth[0])-127
        read_value_y = int(read_bluetooth[1])-127
        read_value_z = int(read_bluetooth[2])-127
    except:
        pass
    bend = 0
    try:
        pressure1 = read_bluetooth[3] #食指
        pressure2 = read_bluetooth[4]
        bend = read_bluetooth[5]
    except:
         pass
    if (abs(read_value_x)<3):
         read_value_x = 0
    if (abs(read_value_y)<3):
         read_value_y = 0
    ratio = 1
    if (bend == 1):
        if(last_band==0):
            for k in range(50):
                read_bluetooth=ser.readline()
        sec_x = []
        sec_y = []
        sec_z = []
        for i in range(300):
            read_bluetooth=ser.readline()
            try:
                read_value_x = int(read_bluetooth[0])-127
                read_value_y = int(read_bluetooth[1])-127
                read_value_z = int(read_bluetooth[2])-127
            except:
                pass
            sec_x.append(read_value_x)
            sec_y.append(read_value_y)
            sec_z.append(read_value_z)
        sec_all = [sec_x,sec_y,sec_z]
        for ax in range(3):
            for j in range(300):
                temp = j
                while (sec_all[ax][j]<-80):
                    temp = j-1
                    sec_all[ax][j] = sec_all[ax][temp]

        train_numpy = np.array(sec_all).reshape(([-1,3,300])).astype('float')
        train_numpy = np.transpose(train_numpy, (0, 2, 1))
        for i in range(len(train_numpy)):
            for k in range(3):
                max_num = np.max(train_numpy[i,:,k])
                min_num = np.min(train_numpy[i,:,k])
                if((max_num - min_num)!= 0):
                    train_numpy[i,:,k] = (train_numpy[i,:,k]-min_num)/(max_num - min_num)
                else:
                    train_numpy[i,:,k] = 0
        data = torch.from_numpy(train_numpy).cuda().float()
        with torch.no_grad():
            outputs = model(data)
        print('get',chr(np.argmax(outputs.cpu().detach().numpy()) + 97 ))
        last_band = 1
    elif (pressure1>150 and pressure2>150):
        last_band = 0
        mouse.right_click()
        #print('right')
        for i in range(5):
            read_bluetooth=ser.readline()
    else:
        last_band = 0
        if (pressure1>30 and pressure2>30 ):
            if (read_value_y>5):
                mouse.wheel(1)
               
            else:
                mouse.wheel(-1)
               
               
        elif (pressure1>30):
            mouse.move(-read_value_x*ratio, read_value_y*ratio, absolute=False) #用1.5秒移動到x=100，y=100的位置
        if (pressure1>=150):
            mouse.click()
            for i in range(30):
                read_bluetooth=ser.readline()
           #print('left')
       #else:
           #mouse.release()
           #print('release')
         #mouse.click('left')
         #for i in range(30):
         #     read_bluetooth=ser.readline()
    print(read_value_x,read_value_y,read_value_z,pressure1,pressure2,bend)
    #print(read_value)
    #print(ser.readline().decode("utf-8"))
    #print(read_bluetooth.decode(encoding='UTF-8',errors='strict'),type(read_bluetooth))

In [ ]:
k = 66
a = np.load(base_path+'/'+chr(k)+chr(k)+'.npy')
print(a.shape)
plt.plot(a[15][2])

In [ ]:
for k in range (65,92):
    #alphabet = ord(alphabet)-97
    a = np.load(base_path+'/'+chr(k)+chr(k)+'.npy')
    for ax in range(3):
        for i in range(5,100):
            for j in range(400):
                temp = j
                while (a[i][ax][j]<-60):
                    temp = j-1
                    a[i][ax][j] = a[i][ax][temp]
    for ax in range(4):
        for i in range(5,100):
            min = np.mean(a[i][ax])
            max = np.max(a[i][ax])
            a[i][ax] = (a[i][ax]-min )/(max-min)
    np.save(base_path+'/'+chr(k)+'_r',a)

In [ ]:
a.shape

In [ ]:
a[0][0]


import numpy as np
import matplotlib.pyplot as plt
for ax in range(3):
    for i in range(5,100):
        for j in range(300):
            temp = j
            while (a[i][ax][j]<-80):
                temp = j-1
                a[i][ax][j] = a[i][ax][temp]

In [ ]:
a = np.load('alphabet/c_r.npy')

In [ ]:
a.shape

In [ ]:
thres = 8
thre = []
for k in range(100):
    for ax in range(300):
        if ((a[k][0][ax]**2+a[k][1][ax]**2+a[k][2][ax]**2)**0.5>thres):
            print(ax)
            thre.append(ax)
            break

In [ ]:
k[1]

In [ ]:
for k in range(len(thre)):
    for i in range(15,45):
        plt.plot(a[i][0][thre[i]:thre[i]+195])
plt.show()
for k in range(len(thre)):
    for i in range(15,45):
        plt.plot(a[i][1][thre[i]:thre[i]+195])
plt.show()
for k in range(len(thre)):
    for i in range(15,45):
        plt.plot(a[i][2][thre[i]:thre[i]+195])
plt.show()